In [9]:
import numpy as np
import pandas as pd
import pickle 
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
from sklearn import svm, datasets
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
from sklearn.utils import resample
import seaborn as sn
from scipy.stats import logistic

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.distributions.multivariate_normal as M

from tqdm import tqdm

from sklearn.model_selection import KFold

In [11]:
folder='Mode'
if folder=='Mode':

    data_varlist=['BIKE_HIGH','BUS_HIGH','CAR_HIGH','RIDESHARE_NEW']
    name_corelist=['Bike','Bus','Car','Rideshare']

    Z_var_list=['HH_RACE_MINORITY','FEMALE','POVERTY','MEDCOND_NEW', 'RURAL']
    Z_note_list=['Race','Female','Poverty','Medcond','Rural']
    
else:
    data_varlist=['2WRK_HOME_NEW','2WKRMHM_NEW','PLACE','PRICE']
    name_corelist=['WFH','WFHO','TB','GPI']
    
    Z_var_list=['HH_RACE_MINORITY','FEMALE','POVERTY','MEDCOND_NEW', 'RURAL']
    Z_note_list=['Race','Female','Poverty','Medcond','Rural']
    

In [1]:
datatype='Original'
num_models = 1
n_splits=5

layer=0
learning_rate=1e-4
n_epoch=5000
n_mini_batch=500000

if layer==3:
    model_path ='model_3lyr_'
elif layer==0:
    model_path ='model_base_'


lam_list=[0]


start_time = time.time()

for k in zip(data_varlist,name_corelist):
    run_name =model_path+k[1]#e.g.model_base_Bus
    if folder=='Mode':
        with open('data/NHTS_balanced/'+k[0]+'_2size_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    else:
        with open('data/NHTS_unbalanced/NHTS_'+k[0]+'_W.pkl', 'rb') as f:
            input_data = pickle.load(f)
    X_all_raw=input_data['X_all_raw']
    Y_all=input_data['Y_all']
    
    X_all_raw['FEMALE']=1-X_all_raw['R_SEX_IMP_NEW']
    X_all_raw['RURAL']=1-X_all_raw['URBRUR_NEW']
    
    X_all_raw=X_all_raw.drop(['R_SEX_IMP_NEW','URBRUR_NEW'],axis=1)
    
    for a in zip(Z_var_list,Z_note_list):
        Z_var=a[0]
        DIR = 'results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'/'
        df = []
        for e in range(len(lam_list)):
            p_dic={}
            p_dic['train_accuracy']=[]
            p_dic['test_accuracy']=[]
            p_dic['para_intercept']=[]
            p_dic['para_coef']=[]
            p_dic['train_cm0']=[]
            p_dic['train_cm1']=[]
            p_dic['test_cm0']=[]
            p_dic['test_cm1']=[]
            p_dic['train_FNR_gap']=[]
            p_dic['test_FNR_gap']=[]
            p_dic['train_FPR_gap']=[]
            p_dic['test_FPR_gap']=[]

            p_dic['TRAINING_PROCESS']={}
            p_dic['TRAINING_PROCESS']['best_iter']=[]
            p_dic['TRAINING_PROCESS']['final_iter']=[]
            p_dic['TRAINING_PROCESS']['accuracy']={}
            p_dic['TRAINING_PROCESS']['cost1']={}
            p_dic['TRAINING_PROCESS']['cost2']={}
            p_dic['TRAINING_PROCESS']['accuracy']['training_set']=[]
            p_dic['TRAINING_PROCESS']['accuracy']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost1']['testing_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['training_set']=[]
            p_dic['TRAINING_PROCESS']['cost2']['testing_set']=[]
            


            for i in range(num_models):
                kf = KFold(n_splits, shuffle=True,random_state=0)

                #final results
                accuracy_train_list = []
                accuracy_test_list = []
                
                para_intercept_list=[]
                para_coef_list=[]

                train_cm0_trial=[]
                train_cm1_trial=[]
                test_cm0_trial=[]
                test_cm1_trial=[]

                train_FNR_gap_list=[]
                test_FNR_gap_list=[]
                train_FPR_gap_list=[]
                test_FPR_gap_list=[]

                #training process
                best_iter_trial=[]
                final_iter_trial=[]
                accuracy_training_prcoess_trial=[]
                accuracy_testing_prcoess_trial=[]
                cost1_training_prcoess_trial=[]
                cost1_testing_prcoess_trial=[]
                cost2_training_prcoess_trial=[]
                cost2_testing_prcoess_trial=[]

                con=0
                for train_idx, test_idx in kf.split(X_all_raw,Y_all):
                    con+=1
                    MODEL_NAME = 'lambda'+str(lam_list[e])+'_trial' + str(i)+'_fold'+str(con)

                    X_train_raw = X_all_raw.iloc[train_idx,:]
                    Y_train = Y_all[train_idx]

                    X_test_raw = X_all_raw.iloc[test_idx,:]
                    Y_test = Y_all[test_idx]
                    
                    X_train=X_train_raw.loc[:, X_train_raw.columns != 'WTPERFIN']
                    X_test=X_test_raw.loc[:, X_test_raw.columns != 'WTPERFIN']
                    W_train=X_train_raw['WTPERFIN']
                    W_test=X_test_raw['WTPERFIN']
                    

                    model = LogisticRegression(solver='lbfgs', random_state=i)
                    model.fit(X_train, Y_train,sample_weight=W_train)

                    print(" ")
                    print('Y:',k[1],'; Z:',a[1],' ',MODEL_NAME,' completed')

                    accuracy_train=accuracy_score(Y_train, model.predict(X_train),sample_weight=W_train)
                    accuracy_test=accuracy_score(Y_test, model.predict(X_test),sample_weight=W_test)
                    para_intercept=model.intercept_
                    para_coef=model.coef_
                    print('accuracy_train: ',accuracy_train,' ;accuracy_test: ',accuracy_test)
#                     print('intercept: ',para_intercept,' ;coef: ',para_coef)


                    #save results
            
                    accuracy_train_list.append(accuracy_train)
                    accuracy_test_list.append(accuracy_test)
                    
                    para_intercept_list.append(para_intercept)
                    para_coef_list.append(para_coef)
                    
                    X_train_z1=X_train[X_train[Z_var]==0]
                    X_train_z0=X_train[X_train[Z_var]==1]
                    Y_train_z1=Y_train[X_train_z1.index]
                    Y_train_z0=Y_train[X_train_z0.index]
                    W_train_z1=W_train[X_train_z1.index]
                    W_train_z0=W_train[X_train_z0.index]
                    
                    X_test_z1=X_test[X_test[Z_var]==0]
                    X_test_z0=X_test[X_test[Z_var]==1]
                    Y_test_z1=Y_test[X_test_z1.index]
                    Y_test_z0=Y_test[X_test_z0.index]
                    W_test_z1=W_test[X_test_z1.index]
                    W_test_z0=W_test[X_test_z0.index]

                    train_cm0=confusion_matrix(Y_train_z0, model.predict(X_train_z0),sample_weight=W_train_z0).ravel()
                    train_cm1=confusion_matrix(Y_train_z1, model.predict(X_train_z1),sample_weight=W_train_z1).ravel()
                    train_cm0_trial.append(train_cm0)
                    train_cm1_trial.append(train_cm1)

                    test_cm0=confusion_matrix(Y_test_z0, model.predict(X_test_z0),sample_weight=W_test_z0).ravel()
                    test_cm1=confusion_matrix(Y_test_z1, model.predict(X_test_z1),sample_weight=W_test_z1).ravel()
                    test_cm0_trial.append(test_cm0)
                    test_cm1_trial.append(test_cm1)

                    train_FNR0=train_cm0[2]/(train_cm0[2]+train_cm0[3])
                    train_FNR1=train_cm1[2]/(train_cm1[2]+train_cm1[3])
                    train_FNR_gap=train_FNR0-train_FNR1


                    test_FNR0=test_cm0[2]/(test_cm0[2]+test_cm0[3])
                    test_FNR1=test_cm1[2]/(test_cm1[2]+test_cm1[3])
                    test_FNR_gap=test_FNR0-test_FNR1
                    
                    train_FPR0=train_cm0[1]/(train_cm0[0]+train_cm0[1])
                    train_FPR1=train_cm1[1]/(train_cm1[0]+train_cm1[1])
                    train_FPR_gap=train_FPR0-train_FPR1

                    test_FPR0=test_cm0[1]/(test_cm0[0]+test_cm0[1])
                    test_FPR1=test_cm1[1]/(test_cm1[0]+test_cm1[1])
                    test_FPR_gap=test_FPR0-test_FPR1

                    train_FNR_gap_list.append(train_FNR_gap)
                    test_FNR_gap_list.append(test_FNR_gap)
                    train_FPR_gap_list.append(train_FPR_gap)
                    test_FPR_gap_list.append(test_FPR_gap)
                    
                    print('train_FNR_gap: ',train_FNR_gap,' ;test_FNR_gap: ',test_FNR_gap)
                    print('train_FPR_gap: ',train_FPR_gap,' ;test_FPR_gap: ',test_FPR_gap)
                    time_dur=round(time.time() - start_time, 3)
                    print("--- %s seconds ---" % (time.time() - start_time))



#                     print(n_samples)
                    df.append([lam_list[e], i,con,layer, \
                    accuracy_train, accuracy_test,\
                    train_FNR_gap,test_FNR_gap,train_FPR_gap,test_FPR_gap])




                p_dic['train_accuracy'].append(accuracy_train_list)
                p_dic['test_accuracy'].append(accuracy_test_list)

                p_dic['para_intercept'].append(para_intercept_list)
                p_dic['para_coef'].append(para_coef_list)

                p_dic['train_cm0'].append(train_cm0_trial)
                p_dic['train_cm1'].append(train_cm1_trial)
                p_dic['test_cm0'].append(test_cm0_trial)
                p_dic['test_cm1'].append(test_cm1_trial)
                p_dic['train_FNR_gap'].append(train_FNR_gap_list)
                p_dic['test_FNR_gap'].append(test_FNR_gap_list)
                p_dic['train_FPR_gap'].append(train_FPR_gap_list)
                p_dic['test_FPR_gap'].append(test_FPR_gap_list)



                with open('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'.pkl', 'wb') as f:
                    pickle.dump(p_dic, f)



                df = pd.DataFrame(df, columns = ['lambda','trial','fold','#layers','accuracy_train','accuracy_test',\
                                                 'FNR_gap_train','FNR_gap_test','FPR_gap_train','FPR_gap_test'])
                df.to_csv('results/' + folder+'/'+ datatype+'/'+ a[1]+'/'+run_name+'.csv')